# Extract Transform Load (ETL)

## Purpose
- CovidTracking API is a very expansive dataset with current data that begins early January 2020, however it does not include cases by day or cumulative deaths, both of which are in the FinnHub API data set. Pertinent data from the CovidTracking API are in columns 'date', 'state', 'positive', 'negative', 'deathIncrease'. It is useful for later data analysis to separate and include all of these parameters into one database. 

## Project Steps:
- Import dataset from CovidTracking.com API, export/write to CSV, import CSV
- Import dataset from Finnhub API, export/write to CSV, import CSV
- Using modules of functions
    - Modules get dataset dtypes, statistics, NaN counts and length, before and after cleaning to verify changes in data
        - getTypes, 
        - describeData
        - analyzeNaNs
- Observe the two datasets, 
    CovidTracking API is a dataset over time and FinnHub API only contains current date data
- Clean CovidTracking dataset, drop rows that are not current date and Finnhub dataset to match data
- Remove unneeded Columns
- Convert/Create Column Finnhub dataset column state to state abbreviation to merge
- Merge cleaned dataframes CovidTrackingDF and FinnHubDF on States
- Export data to MongoDB or PostgreSQL, See sql file
- Run project file daily to append new days data to database and create a history

## Project Team:
- Cynthia Zhang
- Temitayo David Olanbiwonnu
- Jen S/phi-6180
- Kent Thomas

--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------

# <! Module Creation
- The next three cells create modules that can be imported to analyze Dtypes, NaNs, and Describes the data
- Be sure to UNCOMMENT them when first running the notebook, ie remove hastags from all lines with exception of the lines prefaced by Comment and two hashtags from %%writefile

In [1]:
# #%%writefile getTypes.py
# # Comment: Write file creates a module that can be imported with dependencies, %%writefile -a getTypes.py, remove if func is changed

# import pandas as pd
# import numpy as np
# import requests
# import os
# import json
# import matplotlib.pyplot as plt
# from IPython.core.display import HTML
# from datetime import date, datetime

# #getTypes analyzes a DataFrame's column types
# def getTypes(dataFrameName): #0
#     print('Executing getTypes...')
#     print('------------------')
#     dtypesSeries = dataFrameName.dtypes #1
#     dtypesDF = dtypesSeries.to_frame().reset_index() #2
#     dtypesDF = dtypesDF.rename(columns={"index": "ColumnName", 0: "DataType"}) #3
#     columnNames = dtypesDF.columns.tolist() #4
#     #print(columnNames) #5
#     newColumns = dtypesDF['DataType'].unique().tolist() #6
#     dtypesList = [] #7
#     for i in newColumns:
#         dtypesList.append(str(i))
#     print('Unique values in dtypesDF::') #8
#     print('---------------------------')
#     for i in dtypesList: 
#         print(i)
#     print('---------------------------')
#     print(f'Lenght of dtypesDF:: {len(dtypesDF)}') #9
#     print('---------------------------')
#     dtypesDict = {dtypesDF['ColumnName'][i]: dtypesDF['DataType'][i] for i in range(len(dtypesDF['ColumnName']))}
#     dictSlice = dict(list(dtypesDict.items())[0: 5]) #11
#     #print(dictSlice)
#     counter = 0 #12
#     dtypesGlobalsList = dtypesList
#     for i in dtypesList: #13
#         print(f'Global DataFrame Created:: {i}') #14
#         dtypesGlobalsList = dtypesDF.loc[(dtypesDF['DataType'] == i)] #15
#         globals()[i] = dtypesGlobalsList #16
#         globals()[i] = globals()[i].drop(['DataType'], axis=1).reset_index() #17
#         globals()[i] = globals()[i].rename(columns={"ColumnName": i}) #18
#         globals()[i] = globals()[i].sort_values(i) #19
#         counter += 1
#         #display(HTML(globals()[i].to_html())) #20
#     dtypesSummaryDF = pd.concat([int64, object, float64], axis=1, sort=False) #21
#     columnName = dtypesSummaryDF.columns.tolist() #22
#     #print(columnName) #23
#     counter = 0
#     removeIndex = []
#     for i in columnName: #24
#         if i != 'index':
#             removeIndex.append(columnName.pop(counter))
#             counter += 1
#     #print(removeIndex)
#     dtypesSummaryDF.drop(columns = removeIndex, inplace=True) #25
#     print('---------------------------')
#     print(f'DataFrame Types::')
#     print('---------------------------')
#     display(HTML(dtypesSummaryDF.to_html())) #26
    
# # <! getTypes( ):
# # /0/ getTypes analyzes a DataFrame's column types
# # /1/ Get dtypes as series
# # /2/ Make dataframe out of dtypes, output is messy next steps to format
# # /3/ Rename columns
# # /4/ Get column names list
# # /5/ Print column names list
# # /6/ DataFrame is long and uncompressed attempting to create a smaller one with column names as rows
# # /7/ Setting up list to get unique data types
# # /8/ Printing out the unique data types for observation
# # /9/ Getting length before manipulation
# # /10/ Creating a Dictionary to Pair values and flip keys/cols and values/rows
# # /11/ Grabbing a slice of the dictionary to confirm format
# # /12/ Setting up variables for lists and counter
# # /13/ For loop to manipulate dtypesList into Global Variables with individual dataframes
# # /14/ Print statement to confirm globals creation
# # /15/ Matching the data to the dtypesList/dtypesGlobalList into each new global variable
# # /16/ Defining globals into dataframes
# # /17/ Dropping the Datatype column, keeping only Column Name from Original DF
# # /18/ Renaming the ColumnName column to match the global variable name
# # /19/ Sorting the ColumnName names alphabetically
# # /20/ Displaying each dataframe as HTML within Jupyter//must have import statment in dependencies
# # /21/ Merging/Concatenating global dataframes into one Dataframe
# # /22/ Creating list of column names to remove extra indicies
# # /23/ Printing list to confirm
# # /24/ For loop to pop instances that do not equal 'Index'// This is important because there may be many more dtypes in globals()[i] and there must be duplicates in list to drop all at once
# # /25/ Dropping removeIndex list
# # /26/ Displaying Final DF with all original DF column names as Html under their datatype
# # Comment: by ph1-6180
    

In [2]:
# #%%writefile describeData.py
# # Comment: Write file creates a module that can be imported with dependencies, %%writefile -a describeData.py appends, remove if func is changed
# # Comment: This function prints stats for strings and integer value columns
# import pandas as pd
# import numpy as np
# import requests
# import os
# import json
# import matplotlib.pyplot as plt
# from IPython.core.display import HTML
# from datetime import date, datetime

# def describeData(dataFrameName):
#     print('Executing describeData...')
#     print('-------------------------------')
#     global keyHeaders, colsData, stringDescribe, intDescribe, keyStr, KeyInt
#     keyStr, keyInt, keyHeaders, intDescribe, stringDescribe = [], [], [], [], []
#     for key, value in dataFrameName.items():
#         #grabs cols as keys into list
#         keyHeaders.append(key)
#     for i in keyHeaders:
#         #checks the cols data if string
#         if isinstance(dataFrameName[i][0], (str)):
#             stringDescribe.append(dataFrameName[keyHeaders][i].describe())
#         else:
#             intDescribe.append(dataFrameName[keyHeaders][i].describe())
#     stringDescribe = pd.DataFrame.from_dict(dict(zip(keyHeaders, stringDescribe)), orient='index')
#     intDescribe = pd.DataFrame.from_dict(dict(zip(keyHeaders, intDescribe)), orient='index') 
#     #adding pretty print to dataframes, don't forget import statment when copying code
#     print('-------------------------------')
#     print('Object Describe Dataframe')
#     print('-------------------------------')
#     display(HTML(stringDescribe.to_html()))
#     #print(stringDescribe)
#     print('-------------------------------')
#     print('Integer/FloatDescribe Dataframe')
#     print('-------------------------------')
#     display(HTML(intDescribe.to_html()))
#     #print(intDescribe)
#     lengthofDF = len(dataFrameName)
#     print('-------------------------------')
#     print(f'Dataframe Length: {lengthofDF}')
#     print('-------------------------------')
#     columnNames = dataFrameName.columns.tolist()
#     print(f'ColumnNames: \n{columnNames}')
# # Comment: by ph1-6180

In [3]:
# #%%writefile analyzeNaNs.py
# # Comment: Write file creates a module that can be imported with dependencies, %%writefile -a analyzeNaNs.py appends, remove if func is changed
# # Comment: This function analyzes the NaN's in a DF
# # Comment: Print/Returns a Dataframe with the NaN's count and the list of columns without NaN's
# import pandas as pd
# import numpy as np
# import requests
# import os
# import json
# import matplotlib.pyplot as plt
# from IPython.core.display import HTML
# from datetime import date, datetime

# def analyzeNaNs(dataFrameName):
#     print('Executing analyzeNaNs...')
#     print('---------------------')
#     columnNames = dataFrameName.columns.tolist()
#     NaNslist = []
#     noNaNs =[]
#     counter = 0
#     for i in columnNames:
#         colNaNs = dataFrameName[i].isna().sum()
#         NaNslist.append(colNaNs)
#         #print(f'{colNaNs} NaNs in {columnNames[counter]}')
#         counter += 1
#     #print(NaNslist)
#     #print(columnNames)
#     NaNsDF = pd.DataFrame(NaNslist, index = columnNames, columns =['NaNsCount'])
#     transposeDF = NaNsDF.T
#     for i in columnNames:
#         if transposeDF[i][0] == 0:
#             noNaNs.append(i)
#             transposeDF = transposeDF.drop([i], axis=1)
#     print('----------------------')
#     print('Columns NoNaNs::')
#     print('----------------------')
#     for j in range(len(noNaNs)):
#         alphaCols = sorted(noNaNs)
#         print(f'{alphaCols[j]}')
#     print('\n\n\n---------------------')
#     print('NaNs Count DataFrame::')
#     print('---------------------')
#     display(HTML(transposeDF.to_html()))
#     #print(transposeDF)
# # Comment: by ph1-6180

--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------

# <! Import Dependencies

In [4]:
# Dependencies
import getTypes
import describeData
import analyzeNaNs
import pandas as pd
import numpy as np
import requests
import os
import json
import matplotlib.pyplot as plt
from IPython.core.display import HTML
from datetime import date, datetime
#Kent Thomas, ph1-618O

--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------

# <! Query Covid Tracking API

In [5]:
# Pathway to use later to export CSV for cleaning.
covidTrackingDataPath = ".Resources/daily_covid_tracking.csv"

# Country input statement to search for api
# country= input("Type a country you wish to search for: ")

# URL for the API call
covidTrackingDataPath = f"https://api.covidtracking.com/v1/states/daily.json"

# Calling API and printing response.
response = requests.get(covidTrackingDataPath).json()


In [7]:
covidTrackingDF = pd.DataFrame(response)
covidTrackingDF

,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200925,AK,8202.0,433130.0,NaN,441332.0,43.0,NaN,NaN,NaN,...,441332,6,0,0271fcc794329b2dd035402cb74aad56124c446b,0,0,0,0,0,
1,20200925,AL,150658.0,963364.0,NaN,1097595.0,718.0,16852.0,NaN,1782.0,...,1114022,-15,74,08184faa52c95520b2871a34cbea8adad1c1d054,0,0,0,0,0,
2,20200925,AR,79946.0,848822.0,NaN,926294.0,478.0,5202.0,224.0,NaN,...,928768,20,42,6d09ffffc7c9131f7489f7d8e3e4501ce55b8e1e,0,0,0,0,0,
3,20200925,AS,0.0,1571.0,NaN,1571.0,NaN,NaN,NaN,NaN,...,1571,0,0,84c6296997b54d1474da9d9c51e33d48fda7fb5b,0,0,0,0,0,
4,20200925,AZ,216367.0,1211657.0,NaN,1423603.0,521.0,21972.0,119.0,NaN,...,1428024,28,30,a7a38e5509bf8d1ebfe03d1d9acd5febad3546d0,0,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11517,20200124,WA,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,...,0,0,0,6f40087f42d06db4121e09b184785b4110cd4df8,0,0,0,0,0,
11518,20200123,MA,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,0,0,0,885628de5b5c6da109b79adb7faad55e4815624a,0,0,0,0,0,
11519,20200123,WA,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,...,0,0,0,978c05d8a7a9d46e9fa826d83215f5b9732f2c6d,0,0,0,0,0,
11520,20200122,MA,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,0,0,0,0f3eebd5c4a00d0aaa235b0534bd4243794652b6,0,0,0,0,0,


--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------

# <! Exporting CovidTracking CSV
- Covid Tracking Data json to csv

In [9]:
outputPath = os.path.join(".", "Resources", "covidTrackingCurrent.csv")
covidTrackingDF.to_csv(outputPath, index = False)
#Kent Thomas

--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------

# <! Importing CovidTracking CSV
- Read back in Covid Tracking Data CSV for Cleaning

In [10]:
covidTrackingDF = pd.read_csv(outputPath)
covidTrackingDF
#Kent Thomas

,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200925,AK,8202.0,433130.0,NaN,441332.0,43.0,NaN,NaN,NaN,...,441332,6,0,0271fcc794329b2dd035402cb74aad56124c446b,0,0,0,0,0,NaN
1,20200925,AL,150658.0,963364.0,NaN,1097595.0,718.0,16852.0,NaN,1782.0,...,1114022,-15,74,08184faa52c95520b2871a34cbea8adad1c1d054,0,0,0,0,0,NaN
2,20200925,AR,79946.0,848822.0,NaN,926294.0,478.0,5202.0,224.0,NaN,...,928768,20,42,6d09ffffc7c9131f7489f7d8e3e4501ce55b8e1e,0,0,0,0,0,NaN
3,20200925,AS,0.0,1571.0,NaN,1571.0,NaN,NaN,NaN,NaN,...,1571,0,0,84c6296997b54d1474da9d9c51e33d48fda7fb5b,0,0,0,0,0,NaN
4,20200925,AZ,216367.0,1211657.0,NaN,1423603.0,521.0,21972.0,119.0,NaN,...,1428024,28,30,a7a38e5509bf8d1ebfe03d1d9acd5febad3546d0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11517,20200124,WA,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,...,0,0,0,6f40087f42d06db4121e09b184785b4110cd4df8,0,0,0,0,0,NaN
11518,20200123,MA,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,0,0,0,885628de5b5c6da109b79adb7faad55e4815624a,0,0,0,0,0,NaN
11519,20200123,WA,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,...,0,0,0,978c05d8a7a9d46e9fa826d83215f5b9732f2c6d,0,0,0,0,0,NaN
11520,20200122,MA,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,0,0,0,0f3eebd5c4a00d0aaa235b0534bd4243794652b6,0,0,0,0,0,NaN


In [11]:
#Running Module getTypes, analyzeNans and describeData to analyze DF types
#Be sure to uncomment getTypes and %%writefile and execute before importing
#Additionally recomment after executing
#getTypes.getTypes(covidTrackingDF)
#analyzeNaNs.analyzeNaNs(covidTrackingDF)
#describeData.describeData(covidTrackingDF)

In [12]:
print(type(covidTrackingDF))
covidTrackingDF.head()

<class 'pandas.core.frame.DataFrame'>


,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200925,AK,8202.0,433130.0,NaN,441332.0,43.0,NaN,NaN,NaN,...,441332,6,0,0271fcc794329b2dd035402cb74aad56124c446b,0,0,0,0,0,NaN
1,20200925,AL,150658.0,963364.0,NaN,1097595.0,718.0,16852.0,NaN,1782.0,...,1114022,-15,74,08184faa52c95520b2871a34cbea8adad1c1d054,0,0,0,0,0,NaN
2,20200925,AR,79946.0,848822.0,NaN,926294.0,478.0,5202.0,224.0,NaN,...,928768,20,42,6d09ffffc7c9131f7489f7d8e3e4501ce55b8e1e,0,0,0,0,0,NaN
3,20200925,AS,0.0,1571.0,NaN,1571.0,NaN,NaN,NaN,NaN,...,1571,0,0,84c6296997b54d1474da9d9c51e33d48fda7fb5b,0,0,0,0,0,NaN
4,20200925,AZ,216367.0,1211657.0,NaN,1423603.0,521.0,21972.0,119.0,NaN,...,1428024,28,30,a7a38e5509bf8d1ebfe03d1d9acd5febad3546d0,0,0,0,0,0,NaN


In [13]:
# #Checking column indicies
# covidTrackingDF.columns[2]

In [14]:
#Change NaN's to 0's
covidTrackingDF = covidTrackingDF.fillna(0)
#Cynthia Zhang

In [15]:
# Drop all except date, state, postive, negative, deathIncrease
columnNames = covidTrackingDF.columns.tolist()
#print(columnNames)
necessaryDataList = ['date', 'state', 'positive', 'negative', 'deathIncrease']
for i in columnNames:
    if i not in necessaryDataList:
        covidTrackingDF = covidTrackingDF.drop([i], axis = 1)

In [16]:
covidTrackingDF

,date,state,positive,negative,deathIncrease
0,20200925,AK,8202.0,433130.0,6
1,20200925,AL,150658.0,963364.0,-15
2,20200925,AR,79946.0,848822.0,20
3,20200925,AS,0.0,1571.0,0
4,20200925,AZ,216367.0,1211657.0,28
...,...,...,...,...,...
11517,20200124,WA,0.0,0.0,0
11518,20200123,MA,0.0,0.0,0
11519,20200123,WA,0.0,0.0,0
11520,20200122,MA,0.0,0.0,0


In [17]:
#Running Module getTypes, analyzeNans and describeData to analyze DF types
#Be sure to uncomment getTypes and %%writefile and execute before importing
#Additionally recomment after executing
getTypes.getTypes(covidTrackingDF)

Executing getTypes...
------------------
Unique values in dtypesDF::
---------------------------
int64
object
float64
---------------------------
Lenght of dtypesDF:: 5
---------------------------
Global DataFrame Created:: int64
Global DataFrame Created:: object
Global DataFrame Created:: float64
---------------------------
DataFrame Types::
---------------------------


,int64,object,float64
0,date,state,positive
1,deathIncrease,NaN,negative


In [18]:
analyzeNaNs.analyzeNaNs(covidTrackingDF)

Executing analyzeNaNs...
---------------------
----------------------
Columns NoNaNs::
----------------------
date
deathIncrease
negative
positive
state



---------------------
NaNs Count DataFrame::
---------------------


""
NaNsCount


In [19]:
describeData.describeData(covidTrackingDF)

Executing describeData...
-------------------------------
-------------------------------
Object Describe Dataframe
-------------------------------


,count,unique,top,freq
date,11522,56,MA,248


-------------------------------
Integer/FloatDescribe Dataframe
-------------------------------


,count,mean,std,min,25%,50%,75%,max
date,11522.0,2.020061e+07,1.959396e+02,20200122.0,20200424.00,20200615.0,20200805.00,20200925.0
state,11522.0,4.946431e+04,9.902616e+04,0.0,1307.00,12147.5,53520.00,794040.0
positive,11522.0,5.418988e+05,1.169786e+06,0.0,21856.75,154310.5,552175.25,13258207.0
negative,11522.0,1.698455e+01,4.709346e+01,-213.0,0.00,3.0,14.00,951.0


-------------------------------
Dataframe Length: 11522
-------------------------------
ColumnNames: 
['date', 'state', 'positive', 'negative', 'deathIncrease']


In [20]:
#Getting the most current date from Covid Tracking API to merge to FinnHub
#Covid Tracking API orders the most recent data at index 0
#Matching all dates == ['date']Index 0
covidSubset = covidTrackingDF.loc[covidTrackingDF['date'] == covidTrackingDF['date'][0]].copy()
covidSubset.head()

,date,state,positive,negative,deathIncrease
0,20200925,AK,8202.0,433130.0,6
1,20200925,AL,150658.0,963364.0,-15
2,20200925,AR,79946.0,848822.0,20
3,20200925,AS,0.0,1571.0,0
4,20200925,AZ,216367.0,1211657.0,28


--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------

# <! Creating States Dictionary

In [21]:
#Creating US States Dictionary to merge data sets
usStateAbbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'American Samoa': 'AS', 'Arizona': 'AZ',
    'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT',
    'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA',
    'Guam': 'GU', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 
    'Indiana': 'IN','Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY',
    'Louisiana': 'LA', 'Maine': 'ME','Maryland': 'MD', 'Massachusetts': 'MA',
    'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Northern Mariana Islands':'MP', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA','Puerto Rico': 'PR', 'Rhode Island': 'RI',
    'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX',
    'Utah': 'UT', 'Vermont': 'VT', 'Virgin Islands': 'VI', 'Virginia': 'VA',
    'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

In [22]:
#Add tutors code
usStateAbbrev.keys()
usStateAbbrev.values()
usStateDF = pd.DataFrame(usStateAbbrev.keys())
usStateDF['state_abbr'] = pd.DataFrame(usStateAbbrev.values())
usStateDF.reset_index()
usStateDF.head()

,0,state_abbr
0,Alabama,AL
1,Alaska,AK
2,American Samoa,AS
3,Arizona,AZ
4,Arkansas,AR


--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------

# <! Query FinnHub API

In [23]:
# Change full state to abbreviation
# Drop rows 55 - 63, Grand Princess
# Make new column with converted date
# Kent Thomas

In [25]:
# URL for the API call
us_covid_tracking_data = "https://finnhub.io/api/v1/covid19/us"

# Calling API and printing response.
responseFinnhub = requests.get(us_covid_tracking_data).json()
#print(responseFinnhub)
#print(json.dumps(responseFinnhub, indent=4, sort_keys=True)) #printing pretty the response
#Kent Thomas

In [ ]:
responseFinnhubDF = pd.DataFrame(responseFinnhub)
responseFinnhubDF
#Kent Thomas

--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------

# <! Exporting FinnHub CSV

In [ ]:
outputPath = os.path.join(".", "Resources", "FinnhubCovidCurrent.csv")
#outputPath = "./Resources/FinnhubCovidCurrent.csv"
responseFinnhubDF.to_csv(outputPath, index = False)
#Kent Thomas

--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------

# <! Importing FinnHub CSV

In [26]:
# Read back in CSV for Cleaning
FinnHubCSV = pd.read_csv(os.path.join(".", "Resources", "FinnhubCovidCurrent.csv"))
FinnHubCSV
#Kent Thomas

,state,case,death,updated
0,New York,459558,33122,2020-09-27 00:01:25
1,New Jersey,202178,16222,2020-09-27 00:01:25
2,California,804301,15544,2020-09-27 00:01:25
3,Michigan,133377,7030,2020-09-27 00:01:25
4,Florida,698683,14023,2020-09-27 00:01:25
...,...,...,...,...
57,Wuhan Evacuee,4,0,2020-09-27 00:01:25
58,Northern Mariana Islands,31,2,2020-09-27 00:01:25
59,US Military,64602,95,2020-09-27 00:01:25
60,Federal Bureau of Prisons,16447,126,2020-09-27 00:01:25


In [27]:
print(type(FinnHubCSV['updated']))

<class 'pandas.core.series.Series'>


--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------

# <! Merge on State

In [28]:
#Merging usStateDF with FinnHubDF to later merge with covidTrackingDF
addStateAbbrDF = FinnHubCSV.merge(usStateDF, left_on='state', right_on=0)
addStateAbbrDF = addStateAbbrDF.drop([0], axis =1)
#addStateAbbrDF.head(20)

--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------

# <! Final Merge

In [29]:
#Merging covidTracking subset with FinnHub subset on state
combinedDF = covidSubset.merge(addStateAbbrDF, left_on='state', right_on='state_abbr')
combinedDF = combinedDF.drop(['state_x'], axis = 1)
combinedDF = combinedDF.rename(columns={"state_y": "state"})
combinedDF.head(20)

,date,positive,negative,deathIncrease,state,case,death,updated,state_abbr
0,20200925,8202.0,433130.0,6,Alaska,7254,52,2020-09-27 00:01:25,AK
1,20200925,150658.0,963364.0,-15,Alabama,151598,2529,2020-09-27 00:01:25,AL
2,20200925,79946.0,848822.0,20,Arkansas,79946,1266,2020-09-27 00:01:25,AR
3,20200925,216367.0,1211657.0,28,Arizona,216367,5587,2020-09-27 00:01:25,AZ
4,20200925,794040.0,13258207.0,84,California,804301,15544,2020-09-27 00:01:25,CA
5,20200925,67217.0,795588.0,9,Colorado,67504,2030,2020-09-27 00:01:25,CO
6,20200925,56587.0,1462564.0,2,Connecticut,56587,4502,2020-09-27 00:01:25,CT
7,20200925,20085.0,259289.0,1,Delaware,20085,631,2020-09-27 00:01:25,DE
8,20200925,695887.0,4510107.0,122,Florida,698683,14023,2020-09-27 00:01:25,FL
9,20200925,312514.0,2534980.0,52,Georgia,313521,6879,2020-09-27 00:01:25,GA


In [30]:
#Running Module getTypes, analyzeNans and describeData to analyze DF types
#Be sure to uncomment getTypes and %%writefile and execute before importing
#Additionally recomment after executing
getTypes.getTypes(combinedDF)

Executing getTypes...
------------------
Unique values in dtypesDF::
---------------------------
int64
float64
object
---------------------------
Lenght of dtypesDF:: 9
---------------------------
Global DataFrame Created:: int64
Global DataFrame Created:: float64
Global DataFrame Created:: object
---------------------------
DataFrame Types::
---------------------------


,int64,object,float64
0,date,state,positive
1,deathIncrease,updated,negative
2,case,state_abbr,NaN
3,death,NaN,NaN


In [31]:
#This reports if there are any NaN's in DF
# If DataFrame of NaN's is empty there are no NaNs
analyzeNaNs.analyzeNaNs(combinedDF)

Executing analyzeNaNs...
---------------------
----------------------
Columns NoNaNs::
----------------------
case
date
death
deathIncrease
negative
positive
state
state_abbr
updated



---------------------
NaNs Count DataFrame::
---------------------


""
NaNsCount


In [32]:
describeData.describeData(combinedDF)

Executing describeData...
-------------------------------
-------------------------------
Object Describe Dataframe
-------------------------------


,count,unique,top,freq
date,53,53,Massachusetts,1
positive,53,1,2020-09-27 00:01:25,53
negative,53,53,ME,1


-------------------------------
Integer/FloatDescribe Dataframe
-------------------------------


,count,mean,std,min,25%,50%,75%,max
date,53.0,2.020092e+07,0.000000e+00,20200925.0,20200925.0,20200925.0,20200925.0,20200925.0
positive,53.0,1.317166e+05,1.746547e+05,69.0,24181.0,82045.0,148894.0,794040.0
negative,53.0,1.673918e+06,2.356814e+06,14776.0,394573.0,963364.0,1836216.0,13258207.0
deathIncrease,53.0,1.588679e+01,2.526221e+01,-15.0,2.0,7.0,19.0,122.0
state,53.0,1.323736e+05,1.766618e+05,31.0,24034.0,82521.0,148897.0,804301.0
case,53.0,3.832170e+03,5.873131e+03,2.0,451.0,1573.0,4502.0,33122.0


-------------------------------
Dataframe Length: 53
-------------------------------
ColumnNames: 
['date', 'positive', 'negative', 'deathIncrease', 'state', 'case', 'death', 'updated', 'state_abbr']


--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------

# <! Exporting CombinedDF CSV

In [33]:
outputPath = os.path.join(".", "Resources", "combinedCovidData.csv")
#outputPath = "./Resources/covidTrackingCurrent.csv"
combinedDF.to_csv(outputPath, index = False)

In [34]:
#Export to CombinedDF to Mongo DB or SQLite

In [35]:
# #Comment formatting
# for i in range(0, 26):
#     print("- /" + str(i) +'/ ')

--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------

# <! Appending Days Query to Database
- Execute daily to append new days data for larger history
- Code necessary to create a larger database 

In [36]:
# #Reading in exported csv
# csv_file = "./Resources/combinedCovidData.csv"
# combinedDFCSV = pd.read_csv(csv_file)
# combinedDFCSV.head()

In [37]:
# #Connect to local database
# #Original code use to edit username/password 
# #rds_connection_string = "<insert user name>:<insert password>@localhost:5432/customer_db"
# rds_connection_string = "postgres@localhost:5432/databasename"
# engine = create_engine(f'postgresql://{rds_connection_string}')

In [38]:
# #Check for Tables
# engine.table_names()

In [39]:
# #### Use pandas to load csv converted DataFrame into database
# combinedDFCSV.to_sql(name='databasename', con=engine, if_exists='append', index=False)

In [40]:
# ### Confirm data has been added by querying the customer_name table
# #* NOTE: can also check using pgAdmin
# pd.read_sql_query('select * from databasename', con=engine).head()

--------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------

# <! Converting Notebook to Py

In [41]:
# Pip install brings in jupyternotebook converter to Python File
# Will reply with 'Requirement already satisfied' after first install
# Exporting jupyternotebook file to .py

In [42]:
%%bash
pip install nbconvert
jupyter nbconvert --to script ETL-queries.ipynb

[NbConvertApp] Converting notebook ETL-queries.ipynb to script
[NbConvertApp] Writing 24280 bytes to ETL-queries.py


In [ ]:
## ph1-6180

# !> Summation & Analysis
- For this ETL(Extract Transform Load) application the API's chosen are to represent COVID-19 cases current data and exported into a sql database
- Utilized Covid Tracking Project API and FinnHub at:
    - https://covidtracking.com/data/api
    - https://covidtracking.com/data/api


# !> Further Considerations
- Connecting ETL-queries.ipynb as .py to SQLite/SQLALCHEMY
- Connecting combinedcovidDF to mongoDB using pymongo
- Creating Bar Graph visualizations
- Apply a T-Test to the Covid Tracking Project API response and Finnhub response to compare the populations and determine if the populations have identical variances by default for cases and positive. 
    - (The test measures whether the average (expected) value differs significantly the two datasets. If there is a large p-value, for example larger than 0.05 or 0.1, then the populations are similar. If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then the populations vary.
    - More information about T-Test
        - https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
- Finnhub Rate Limits
    Receive a response with status code 429 if limits exceeded
    On top of all plan's limit, there is a 30 API calls/ second limit.


# !> Resources
- The first data set comes from the COVID Tracking Project, an organization formed by 'The Atlanic' a newspaper and online resource for news. They collect COVID data from all 50 states and territories daily. COVID data from the Covid Tracking Project has been utilized by the White House, and many other popular news networks.
- Second data set comes from Finnhub.io which sources the CDC and reputable sources.


Links to Covid Tracking Project API & Documentation:
- https://covidtracking.com/data/api
- https://documenter.getpostman.com/view/8854915/SzS8rjHv?version=latest

Link to FinnHub Documentation: 
- https://finnhub.io/api/v1/covid19/us
- https://finnhub.io/docs/api#covid-19
- https://documenter.getpostman.com/view/10724784/SzYW3LFa?version=latest


# !> Credits
Sarah Popelka, Latisha McNeel, Paulo Ramos and Reed Hyde assisted with debugging. (Additionally much credit must be given to a third party that assisted with suggesting the Postman website.)